In [1]:
import pandas as pd

df = pd.read_csv('combined_data.csv')
print("Headers of combined_data.csv:")
print(df.columns.tolist())


Headers of combined_data.csv:
['datetime', 'name', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'preciptype', 'windgust', 'windspeed', 'cloudcover', 'visibility', 'uvindex', 'sunrise', 'sunset', 'conditions', 'description', 'icon', 'Close/Last', 'Volume', 'Open', 'High', 'Low', 'GasPrice']


In [22]:
import altair as alt


df = pd.read_csv('combined_data.csv', parse_dates=['datetime'])

date_range = alt.selection_interval(encodings=['x'])


click = alt.selection_point(
    fields=['datetime'],
    nearest=True,
    on='click',
    empty='none',
    clear='dblclick'
)


base = alt.Chart(df).encode(
    x=alt.X('datetime:T', title='Date')
)

stock_line = base.mark_line(color='blue').encode(
    y=alt.Y('Close/Last:Q', title='Tesla Stock Price ($)')
).properties(
    width=800,
    height=300
).add_selection(date_range)


temp_line = base.mark_line(color='red').encode(
    y=alt.Y('temp:Q', title='Temperature (°F)', axis=alt.Axis(orient='right'))
)


overlay = base.mark_point(opacity=0).encode(
    x='datetime:T'
).add_params(click)


rule = alt.Chart(df).mark_rule(color='gray').encode(
    x='datetime:T'
).transform_filter(click)

text = alt.Chart(df).mark_text(align='left', dx=5, dy=-5, fontSize=12).encode(
    x='datetime:T',
    y='Close/Last:Q',
    text=alt.Text('info:N')
).transform_calculate(
    info=' "Date: " + timeFormat(datum.datetime, "%Y-%m-%d") + ", Close: " + datum["Close/Last"] + ", Temp: " + datum.temp'
).transform_filter(click)


layered_chart = alt.layer(
    stock_line,
    temp_line,
    overlay,
    rule,
    text
).resolve_scale(
    y='independent'
).properties(
    title='Tesla Stock Price vs Temperature Over Time'
).interactive()

layered_chart

layered_chart.save('tesla_temp_chart.png')

/var/folders/pv/6dqfzdqd4y1fp3_lh4hw7lm40000gn/T/ipykernel_12247/3131616512.py:22: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  stock_line = base.mark_line(color='blue').encode(


ValueError: Saving charts in 'png' format requires the vl-convert-python package: see https://altair-viz.github.io/user_guide/saving_charts.html#png-svg-and-pdf-format

In [14]:

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
corr_matrix = df[numeric_cols].corr()


corr_melt = corr_matrix.reset_index().melt(id_vars='index')
corr_melt.columns = ['Variable1', 'Variable2', 'Correlation']


heatmap = alt.Chart(corr_melt).mark_rect().encode(
    x=alt.X('Variable1:N', title='Variable 1', sort=None),
    y=alt.Y('Variable2:N', title='Variable 2', sort=None),
    color=alt.Color('Correlation:Q', scale=alt.Scale(scheme='viridis')),
    tooltip=[
        alt.Tooltip('Variable1:N', title='Variable 1'),
        alt.Tooltip('Variable2:N', title='Variable 2'),
        alt.Tooltip('Correlation:Q', title='Correlation', format=".2f")
    ]
).properties(
    width=400,
    height=400,
    title='Correlation Heatmap of Combined Data'
)

heatmap

heatmap.save('correlation_heatmap.html')

In [15]:
import pandas as pd
import altair as alt


df = pd.read_csv('combined_data.csv', parse_dates=['datetime'])


numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
corr_matrix = df[numeric_cols].corr()


corr_melt = corr_matrix.reset_index().melt(id_vars='index')
corr_melt.columns = ['Variable1', 'Variable2', 'Correlation']


tesla_cols = ["Close/Last", "Volume", "Open", "High", "Low"]


filtered_corr = corr_melt[
    ((corr_melt['Variable1'] == 'GasPrice') & (corr_melt['Variable2'].isin(tesla_cols))) |
    ((corr_melt['Variable2'] == 'GasPrice') & (corr_melt['Variable1'].isin(tesla_cols)))
]


heatmap = alt.Chart(filtered_corr).mark_rect().encode(
    x=alt.X('Variable1:N', title='Variable 1', sort=None),
    y=alt.Y('Variable2:N', title='Variable 2', sort=None),
    color=alt.Color('Correlation:Q', 
                    scale=alt.Scale(domain=(-1, 1), scheme='viridis')),
    tooltip=[
        alt.Tooltip('Variable1:N', title='Variable 1'),
        alt.Tooltip('Variable2:N', title='Variable 2'),
        alt.Tooltip('Correlation:Q', title='Correlation', format=".2f")
    ]
).properties(
    width=300,
    height=300,
    title='Correlation Heatmap: GasPrice vs Tesla Stock Metrics'
)

heatmap
# you can choose which one u want

heatmap.save('correlation_heatmap_gasprice.html')

In [16]:
import pandas as pd
import altair as alt


df = pd.read_csv('combined_data.csv', parse_dates=['datetime'])


brush = alt.selection_interval()


# - Temperature on the x-axis,
# - Tesla stock price on the y-axis,
# - Bubble size representing trading volume,
# - Conditional coloring based on the brush selection,
# - Tooltips displaying Date, Temperature, Stock Price, and Volume.
scatter = alt.Chart(df).mark_circle().encode(
    x=alt.X('temp:Q', title='Temperature (°F)'),
    y=alt.Y('Close/Last:Q', title='Tesla Stock Price ($)'),
    size=alt.Size('Volume:Q', title='Trading Volume', scale=alt.Scale(range=[10, 1000])),
    color=alt.condition(brush, alt.value('steelblue'), alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('datetime:T', title='Date'),
        alt.Tooltip('temp:Q', title='Temperature'),
        alt.Tooltip('Close/Last:Q', title='Stock Price'),
        alt.Tooltip('Volume:Q', title='Volume')
    ]
).add_selection(
    brush
)


regression_line = scatter.transform_regression(
    'temp', 'Close/Last', method='linear'
).mark_line(color='red')


chart = (scatter + regression_line).properties(
    width=600,
    height=400,
    title='Scatter Plot: Temperature vs Tesla Stock Price with Regression Overlay'
)

chart

chart.save('temp_scatter_plot.html')

/var/folders/pv/6dqfzdqd4y1fp3_lh4hw7lm40000gn/T/ipykernel_12247/2542002913.py:16: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  scatter = alt.Chart(df).mark_circle().encode(


In [17]:
import pandas as pd
import altair as alt


df = pd.read_csv('combined_data.csv', parse_dates=['datetime'])


df['temp'] = pd.to_numeric(df['temp'], errors='coerce')
df['Close/Last'] = pd.to_numeric(df['Close/Last'], errors='coerce')
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')

points = alt.Chart(df).mark_circle().encode(
    x=alt.X('temp:Q', title='Temperature (°F)'),
    y=alt.Y('Close/Last:Q', title='Tesla Stock Price ($)'),
    size=alt.Size('Volume:Q', title='Trading Volume', scale=alt.Scale(range=[10, 1000])),
    color=alt.value('steelblue'),  # All points in steelblue
    tooltip=[
        alt.Tooltip('datetime:T', title='Date'),
        alt.Tooltip('temp:Q', title='Temperature'),
        alt.Tooltip('Close/Last:Q', title='Stock Price'),
        alt.Tooltip('Volume:Q', title='Volume')
    ]
).properties(
    width=600,
    height=400,
    title='Scatter Plot: Temperature vs. Tesla Stock Price with Regression Overlay'
)


regression_line = alt.Chart(df).transform_regression(
    'temp',      # independent variable
    'Close/Last',# dependent variable
    method='linear'
).mark_line(color='red').encode(
    x=alt.X('temp:Q', title='Temperature (°F)'),
    y=alt.Y('Close/Last:Q', title='Tesla Stock Price ($)')
)


final_chart = points + regression_line
final_chart
# same for here u can choose i added a regression line but its pretty straight so idk if u want it 

final_chart.save('scatter_plot_temp_vs_tesla.html')

In [19]:
import pandas as pd
import altair as alt


df = pd.read_csv('combined_data.csv', parse_dates=['datetime'])

brush = alt.selection_interval(encodings=['x'])

time_series = alt.Chart(df).mark_line(color='blue').encode(
    x=alt.X('datetime:T', title='Date'),
    y=alt.Y('Close/Last:Q', title='Tesla Stock Price ($)')
).properties(
    width=800,
    height=200,
    title='Time Series: Tesla Stock Price Over Time'
).add_selection(
    brush  
)


scatter = alt.Chart(df).mark_circle(size=60).encode(
    x=alt.X('temp:Q', title='Temperature (°F)'),
    y=alt.Y('Close/Last:Q', title='Tesla Stock Price ($)'),
    color=alt.Color('temp:Q', scale=alt.Scale(scheme='viridis')),
    tooltip=[
        alt.Tooltip('datetime:T', title='Date'),
        alt.Tooltip('temp:Q', title='Temperature'),
        alt.Tooltip('Close/Last:Q', title='Stock Price'),
        alt.Tooltip('Volume:Q', title='Volume')
    ]
).transform_filter(
    brush  
).properties(
    width=800,
    height=300,
    title='Scatter Plot: Temperature vs Tesla Stock Price (Filtered by Time)'
)

# Combine the two charts vertically into a dashboard
dashboard = alt.vconcat(
    time_series,
    scatter
)

dashboard

dashboard.save('dashboard.html')

/var/folders/pv/6dqfzdqd4y1fp3_lh4hw7lm40000gn/T/ipykernel_12247/4165947357.py:9: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  time_series = alt.Chart(df).mark_line(color='blue').encode(


In [20]:
import pandas as pd
import altair as alt

df = pd.read_csv('combined_data.csv', parse_dates=['datetime'])

boxplot = alt.Chart(df).mark_boxplot().encode(

    x=alt.X('temp:Q', bin=alt.Bin(maxbins=10), title='Temperature Bin (°F)'),
    y=alt.Y('Close/Last:Q', title='Tesla Stock Price ($)'),
    tooltip=[
        alt.Tooltip('min(Close/Last):Q', title='Min Price', format=".2f"),
        alt.Tooltip('q1(Close/Last):Q', title='Q1', format=".2f"),
        alt.Tooltip('median(Close/Last):Q', title='Median', format=".2f"),
        alt.Tooltip('q3(Close/Last):Q', title='Q3', format=".2f"),
        alt.Tooltip('max(Close/Last):Q', title='Max Price', format=".2f")
    ]
).properties(
    width=600,
    height=400,
    title='Distribution of Tesla Stock Price by Temperature Bin'
)

boxplot

boxplot.save('distribution_tesla_stock_price.html')